In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import nibabel as nib
import os
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.layers import Dense, Conv3D, MaxPooling3D, BatchNormalization, Dropout, concatenate, Flatten, LeakyReLU
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from pathlib import Path
from DataGenerator import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

2021-07-30 18:19:23.174863: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [26]:
import shutil
#keras.backend.clear_session()

In [27]:
kfold = KFold(n_splits=5, shuffle=True)
alz = pd.read_csv("alzheimers.csv").drop_duplicates()
conv = pd.read_csv("converted.csv").drop_duplicates()
non = pd.read_csv("nonconverted.csv").drop_duplicates()
cont = pd.read_csv("control.csv").drop_duplicates()

for index, row in alz.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        alz = alz.drop(index)


for index, row in cont.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        cont = cont.drop(index)
        
for index, row in alz.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        alz = alz.drop(index)


for index, row in cont.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        cont = cont.drop(index)        
        
posmarker = np.ones(len(alz))
negmarker = np.zeros(len(cont))

alz["marker"] = posmarker
cont["marker"] = negmarker
subjects = alz.append(cont)
ids = subjects[["PET_ID", "MRI_ID"]]
markers = subjects["marker"]


In [28]:
acc_per_fold = []
loss_per_fold = []

fold_no = 1
petinput = Input(shape = (182, 218, 182, 1))
mriinput = Input(shape = (182, 218, 182, 1))
x = Conv3D(16, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform', input_shape = (182, 218, 182, 1))(petinput)
x = MaxPooling3D(pool_size = (2,2,2))(x)
x = BatchNormalization(center=True, scale=True)(x)
x = Dropout(0.2)(x)
x = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform')(x)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)
x = BatchNormalization(center=True, scale=True)(x)
x = Dropout(0.2)(x)
x = Model(inputs = petinput, outputs = x)

y = Conv3D(16, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform', input_shape = (182, 218, 182, 1))(mriinput)
y = MaxPooling3D(pool_size = (2,2,2))(y)
y = BatchNormalization(center=True, scale=True)(y)
y = Dropout(0.2)(y)
y = Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform')(y)
y = MaxPooling3D(pool_size=(2, 2, 2))(y)
y = BatchNormalization(center=True, scale=True)(y)
y = Dropout(0.2)(y)
y = Model(inputs = mriinput, outputs = y)

end = concatenate([x.output, y.output])
z = Flatten()(end)
z = Dense(10, kernel_initializer='he_uniform')(z)
z = LeakyReLU()(z)
z = Dense(8, kernel_initializer='he_uniform')(z)
z = LeakyReLU()(z)
z = Dense(5, kernel_initializer='he_uniform')(z)
z = LeakyReLU()(z)
z = Dense(1, activation = "sigmoid", kernel_initializer='he_uniform')(z)
model = Model(inputs=[x.input, y.input], outputs=z)

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

In [29]:
for train, test in kfold.split(ids, markers):
# ids_train, ids_val, markers_train, markers_val = train_test_split(ids, markers, random_state = 42, stratify = markers, test_size = .25)
#train_dataset = tf.data.Dataset.from_tensor_slices((ids_train, markers_train)).map(grabfile)
#val_dataset = tf.data.Dataset.from_tensor_slices((ids_val, markers_val))
    checkpoint_filepath = 'temp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_freq = "epoch",
        monitor='val_acc',
        mode='max',
        save_best_only=True)

    batch_size = 2
    train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)
    val_generator = DataGenerator(ids.iloc[test], markers.iloc[test], batch_size = batch_size)

    history = model.fit(train_generator, validation_data = val_generator, callbacks=[model_checkpoint_callback], epochs = 5)
    
    scores = model.evaluate(val_generator, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fold_no = fold_no + 1

Epoch 1/5
111/111 [==============================] - 3268s 29s/step - loss: 249.7599 - mae: 0.5144 - acc: 0.4865 - val_loss: 758.1491 - val_mae: 0.6607 - val_acc: 0.3393
Epoch 2/5
111/111 [==============================] - 3255s 29s/step - loss: 119.1526 - mae: 0.2928 - acc: 0.7072 - val_loss: 128.1865 - val_mae: 0.6607 - val_acc: 0.3393
Epoch 3/5
111/111 [==============================] - 3262s 29s/step - loss: 55.4169 - mae: 0.1712 - acc: 0.8288 - val_loss: 203.3060 - val_mae: 0.3393 - val_acc: 0.6607
Epoch 4/5
111/111 [==============================] - 3251s 29s/step - loss: 31.8014 - mae: 0.1261 - acc: 0.8739 - val_loss: 174.0164 - val_mae: 0.3393 - val_acc: 0.6607
Epoch 5/5
111/111 [==============================] - 3260s 29s/step - loss: 10.4512 - mae: 0.0732 - acc: 0.9279 - val_loss: 86.8993 - val_mae: 0.2857 - val_acc: 0.7143
Score for fold 1: loss of 86.89924621582031; mae of 28.57142984867096%
Epoch 1/5
111/111 [==============================] - 3266s 29s/step - loss: 45.3168

In [30]:
model.load_weights(checkpoint_filepath)

mode

In [37]:
for index, row in conv.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        conv = conv.drop(index)


for index, row in non.iterrows():
    if not Path("petregistered/" + str(row["PET_ID"]) + "_registered.nii.gz").exists():
        non = non.drop(index)
        
for index, row in conv.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        conv = conv.drop(index)


for index, row in non.iterrows():
    if not Path("registered/" + str(row["MRI_ID"]) + "_registered.nii.gz").exists():
        non = non.drop(index)        
        
posmarker = np.ones(len(conv))
negmarker = np.zeros(len(non))

conv["marker"] = posmarker
non["marker"] = negmarker
subjects = conv.append(non)
ids = subjects[["PET_ID", "MRI_ID"]]
markers = subjects["marker"]

In [35]:
new = keras.models.load_model("models")

In [36]:
new.evaluate(val_generator)

27/27 [==============================] - 85s 3s/step - loss: 7.7034 - mean_absolute_error: 0.0556 - acc: 0.9444


[7.703395366668701, 0.0555555559694767, 0.9444444179534912]

In [40]:
fold_no = 1
new.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

for train, test in kfold.split(ids, markers):
    new.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

#train_dataset = tf.data.Dataset.from_tensor_slices((ids_train, markers_train)).map(grabfile)
#val_dataset = tf.data.Dataset.from_tensor_slices((ids_val, markers_val))
    checkpoint_filepath = 'temp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_freq = "epoch",
        monitor='val_acc',
        mode='max',
        save_best_only=True)

    batch_size = 2
    train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)
    val_generator = DataGenerator(ids.iloc[test], markers.iloc[test], batch_size = batch_size)

    #model.fit(train_generator, validation_data = val_generator, callbacks=[model_checkpoint_callback], epochs = 5)
    
    scores = new.evaluate(val_generator)


  # Increase fold number
    fold_no = fold_no + 1

16/16 [==============================] - 48s 3s/step - loss: 154322.0469 - mae: 0.3125 - acc: 0.6875


In [41]:
for layer in new.layers:
    if type(layer) == Conv3D:
        layer.trainable = False

In [51]:
fold_no = 1
model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

for train, test in kfold.split(ids, markers):
    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["mae", "acc"])

#train_dataset = tf.data.Dataset.from_tensor_slices((ids_train, markers_train)).map(grabfile)
#val_dataset = tf.data.Dataset.from_tensor_slices((ids_val, markers_val))
    checkpoint_filepath = 'temp/checkpoint'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        save_freq = "epoch",
        monitor='val_acc',
        mode='max',
        save_best_only=True)

    batch_size = 2
    train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = batch_size)
    val_generator = DataGenerator(ids.iloc[test], markers.iloc[test], batch_size = batch_size)

    new.fit(train_generator, validation_data = val_generator, callbacks=[model_checkpoint_callback], epochs = 5)
    
    scores = model.evaluate(val_generator, verbose = False)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number
    fold_no = fold_no + 1

Epoch 1/5
64/64 [==============================] - 1875s 29s/step - loss: 9.6936 - mae: 0.0469 - acc: 0.9531 - val_loss: 9.1448 - val_mae: 0.2184 - val_acc: 0.7812
Epoch 2/5
64/64 [==============================] - 1861s 29s/step - loss: 5.2576 - mae: 0.0547 - acc: 0.9453 - val_loss: 22.5297 - val_mae: 0.2802 - val_acc: 0.7188
Epoch 3/5
64/64 [==============================] - 1861s 29s/step - loss: 6.8178 - mae: 0.0234 - acc: 0.9766 - val_loss: 59.9417 - val_mae: 0.3431 - val_acc: 0.6562
Epoch 4/5
64/64 [==============================] - 1863s 29s/step - loss: 1.7852 - mae: 0.0156 - acc: 0.9844 - val_loss: 29.6867 - val_mae: 0.4062 - val_acc: 0.5938
Epoch 5/5
64/64 [==============================] - 1879s 29s/step - loss: 1.4001 - mae: 0.0288 - acc: 0.9688 - val_loss: 10.3994 - val_mae: 0.1370 - val_acc: 0.8750
Score for fold 1: loss of 110.10718536376953; mae of 40.625%
Epoch 1/5
64/64 [==============================] - 1877s 29s/step - loss: 7.7089e-04 - mae: 7.3408e-04 - acc: 1.000

KeyboardInterrupt: 

In [49]:
new.evaluate(val_generator)

16/16 [==============================] - 49s 3s/step - loss: 36.5972 - mae: 0.3750 - acc: 0.6250


[36.59720993041992, 0.37501227855682373, 0.625]

In [52]:
new.save("haha")

INFO:tensorflow:Assets written to: haha/assets


INFO:tensorflow:Assets written to: haha/assets


In [ ]:
img = nib.load("brains/322646.nii.gz")
img = img.get_fdata().squeeze()
print(img.shape)

In [ ]:
import matplotlib.pyplot as plt
def show_slices(slices):
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
slice_0 = img[90, :, :]
slice_1 = img[:, 100, :]
slice_2 = img[:, :, 160]

In [ ]:
show_slices([slice_0, slice_1, slice_2])


In [ ]:
for layer in model.layers:
    print(layer.weights)

In [ ]:
PET = patients.loc[(patients["Modality"] == "PET")]["Image ID"].values


In [ ]:
patients = pd.read_csv("idaSearch_7_07_2021 (3).csv")


In [114]:
model.pop()

AttributeError: 'Functional' object has no attribute 'pop'

In [6]:
train_generator = DataGenerator(ids.iloc[train], markers.iloc[train], batch_size = 1)


NameError: name 'train' is not defined

In [ ]:
print(model(train_generator.__getitem__(30)[0]))

In [11]:
for k in range(20):
    print(train_generator.__getitem__(k)[1])

[[0]
 [0]]
[[0]
 [0]]
[[0]
 [0]]
[[0]
 [0]]
[[1]
 [0]]
[[0]
 [0]]
[[0]
 [0]]
[[1]
 [1]]
[[1]
 [0]]
[[1]
 [0]]
[[1]
 [0]]
[[1]
 [1]]
[[0]
 [1]]
[[0]
 [0]]
[[1]
 [0]]
[[0]
 [1]]
[[0]
 [0]]
[[1]
 [1]]
[[0]
 [1]]
[[0]
 [0]]


In [18]:
for k in range(20):
    print(model(train_generator.__getitem__(k)[0]))

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2028/2259661424.py", line 2, in <module>
    print(model(train_generator.__getitem__(k)[0]))
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 985, in __call__
    outputs = call_fn(inputs, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py", line 386, in call
    inputs, training=training, mask=mask)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py", line 508, in _run_internal_graph
    outputs = node.layer(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 985, in __call__
    outputs = call_fn(inputs, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-pac

TypeError: object of type 'NoneType' has no len()

In [148]:
layer = model.layers[23]

In [178]:
model.layers[20]

In [ ]:
# from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs



In [161]:
outputs

[<tf.Tensor 'input_7:0' shape=(None, 182, 218, 182, 1) dtype=float32>,
 <tf.Tensor 'input_8:0' shape=(None, 182, 218, 182, 1) dtype=float32>,
 <tf.Tensor 'conv3d_12/Relu:0' shape=(None, 180, 216, 180, 16) dtype=float32>,
 <tf.Tensor 'conv3d_14/Relu:0' shape=(None, 180, 216, 180, 16) dtype=float32>,
 <tf.Tensor 'max_pooling3d_12/MaxPool3D:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'max_pooling3d_14/MaxPool3D:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'batch_normalization_12/batchnorm/add_1:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'batch_normalization_14/batchnorm/add_1:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'dropout_12/cond/Identity:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'dropout_14/cond/Identity:0' shape=(None, 90, 108, 90, 16) dtype=float32>,
 <tf.Tensor 'conv3d_13/Relu:0' shape=(None, 88, 106, 88, 32) dtype=float32>,
 <tf.Tensor 'conv3d_15/Relu:0' shape=(None, 88, 106, 88, 32) dtype=fl

In [164]:
intermed = Model(inputs = model.input, outputs = model.layers[22].output)

In [ ]:
for i in range(2):    
    intermed = Model(inputs = model.input, outputs = model.layers[18].output)
    intermediate_output = intermed.predict(val_generator.__getitem__(20)[0])
    print(intermediate_output, flush = True)
    print(i, flush = True)


In [ ]:
intermediate_output